In [2]:
from IPython.display import Image

In [21]:
import os
from dotenv import load_dotenv
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_API_KEY=
# OPENAI_API_KEY=
load_dotenv()

True

## 认识 Concepts 

- 过程信息（所谓的对话式 agent，conversational agents，就是带记忆）
    - stateless LLMs => with memory (maintaining previous interactions)
    - intermediate_steps： list of actions/tools (input, output)
        - agent_scratchpad（草稿本）：list of tool messages
    - 第一次 query-resp 之后，才会有这些过程信息
        - 第一次：`intermediate_steps: []`, `agent_scratchpad: ""`
- chat history
    - messages (openai 的 api)
    - system, user, assistant, user assistant ...
- langsmith: 监控所有的 messages (input, output)

## ReAct (Reasoning & Acting)

- https://arxiv.org/abs/2210.03629
- ReAct（经典、且general、应用广泛的 prompt）
    - User query => (Thought -> Action (Action Input) -> Observation(Action Ouptut)) * N
        - Action input => output, function decision & calling 
    - multi-step/hops reasoning/interacting
    - 跟 function calling（LangChain 中的 tools）天然地适配；

In [6]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [9]:
type(prompt), prompt

(langchain_core.prompts.prompt.PromptTemplate,
 PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'))

In [11]:
# 一个完整的 prompt 模板，历史对话/函数调用（输入输出）信息不断地追加在这一个 prompt 中，而不是维护成 messages list
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


- Thought: you should always think about what to do
- Action: the action to take, should be one of `[{tool_names}]`
- Action Input: the input to the action
- Observation: the result of the action
- ... (this Thought/Action/Action Input/Observation can repeat N times)

## LangChain projects

In [3]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*bkNjqXR1LjAVJQbNLpfYvA.png', width=500)

- AgentExecutor：agent that is using tools

```
next_action = agent.get_action(...)  
while next_action != AgentFinish:
  observation = run(next_action)
  next_action = agent.get_action(..., next_action, observation)
return next_action
```

In [4]:
os.environ["LANGCHAIN_PROJECT"] = 'conversational_agents'

In [17]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent

In [15]:
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001", "Bob": "E002", "Charlie": "E003",
    "Diana": "E004", "Evan": "E005", "Fiona": "E006",
    "George": "E007", "Hannah": "E008", "Ian": "E009",
    "Jasmine": "E010"
  }
  
  return fake_employees.get(name, "Employee not found")

# Custom tool for the Agent 
# ValueError: Function must have a docstring if description not provided.
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000, "E002": 47000, "E003": 52000,
    "E004": 61000, "E005": 45000, "E006": 58000,
    "E007": 49000, "E008": 53000, "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [20]:
prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI(model='gpt-4o')
tools = [get_employee_salary, get_employee_id]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
To determine Evan's salary, I first need to obtain his employee ID.

Action: get_employee_id
Action Input: "Evan"E005Now that I have Evan's employee ID, I can retrieve his salary using this ID.

Action: get_employee_salary
Action Input: E00545000I now know the final answer.

Final Answer: The salary of Evan is 45000.

> Finished chain.


{'input': 'What is the Salary of Evan?',
 'output': 'The salary of Evan is 45000.'}

### first query-resp

LangSmith: https://smith.langchain.com/

 ```
Answer the following questions as best you can. You have access to the following tools:

    get_employee_salary(employee_id) - To get the salary of an employee, it takes employee_id as input and return salary
    get_employee_id(name) - To get employee id, it takes employee name as arguments
    name(str): Name of the employee

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [get_employee_salary, get_employee_id]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: What is the Salary of Evan?
    Thought:
```

```
To determine Evan's salary, I first need to obtain his employee ID.

Action: get_employee_id
Action Input: "Evan"
```

- get_employee_id
    - input: Evan
    - output: E005

### second query-resp


```
Answer the following questions as best you can. You have access to the following tools:

    get_employee_salary(employee_id) - To get the salary of an employee, it takes employee_id as input and return salary
    get_employee_id(name) - To get employee id, it takes employee name as arguments
    name(str): Name of the employee

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [get_employee_salary, get_employee_id]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: What is the Salary of Evan?
    Thought:To determine Evan's salary, I first need to obtain his employee ID.

    Action: get_employee_id
    Action Input: "Evan"
    Observation: E005
    Thought: 
```

```
Now that I have Evan's employee ID, I can retrieve his salary using this ID.

Action: get_employee_salary
Action Input: E005
```

get_employee_salary
- input: E005
- output: 45000

### finally

```
Answer the following questions as best you can. You have access to the following tools:

get_employee_salary(employee_id) - To get the salary of an employee, it takes employee_id as input and return salary
get_employee_id(name) - To get employee id, it takes employee name as arguments
name(str): Name of the employee

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get_employee_salary, get_employee_id]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is the Salary of Evan?
Thought:To determine Evan's salary, I first need to obtain his employee ID.

Action: get_employee_id
Action Input: "Evan"
Observation: E005
Thought: Now that I have Evan's employee ID, I can retrieve his salary using this ID.

Action: get_employee_salary
Action Input: E005
Observation: 45000
Thought: 
```

```
I now know the final answer.

Final Answer: The salary of Evan is 45000.
```

In [22]:
Image(url='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*j5T9A2o7AqQkk6lS', width=500)

### `显式地` 操作 agent_scratchpad 与 intermediate_steps

https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/
- `format_to_openai_tool_messages`
    - https://api.python.langchain.com/en/latest/_modules/langchain/agents/format_scratchpad/openai_tools.html#format_to_openai_tool_messages

In [31]:
os.environ["LANGCHAIN_PROJECT"] = 'memory_agents'

In [32]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [39]:
tools = [get_word_length]

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [40]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [41]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# runnable Sequence，根据需要循环执行这 4 步；
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [42]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
process_list = list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


In [57]:
process_list

[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_P3LQGFQ50JvKMtLpLXJ2SSsZ', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, id='run-94428a05-fa61-4853-8eeb-7c12f68b447e', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_P3LQGFQ50JvKMtLpLXJ2SSsZ', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_P3LQGFQ50JvKMtLpLXJ2SSsZ', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_P3LQGFQ50JvKMtLpLXJ2SSsZ')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_P3LQGFQ50JvKMtLpLXJ

In [50]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [51]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [52]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [53]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, 
                                "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", 
                       "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
"Educa" is not a standard English word. It might be a truncated form of "education" or a name, but it is not commonly recognized as a standalone word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': '"Educa" is not a standard English word. It might be a truncated form of "education" or a name, but it is not commonly recognized as a standalone word in English.'}

- messages
    - system
    - human (user)
    - ai (assistant)
    - human (user)
    - ai (assistant)
    - ...

In [56]:
chat_history = []
# https://smith.langchain.com/hub/hwchase17/openai-functions-agent
# input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history']
prompt = hub.pull('hwchase17/openai-functions-agent')
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
input1 = "how many letters in the word dadebfdr?"
result = agent_executor.invoke({"input": input1, 
                                "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", 
                       "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'dadebfdr'}`


8The word "dadebfdr" has 8 letters.

> Finished chain.


> Entering new AgentExecutor chain...
No, "dadebfdr" does not appear to be a real word in the English language. It seems to be a random string of letters.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word dadebfdr?'),
  AIMessage(content='The word "dadebfdr" has 8 letters.')],
 'output': 'No, "dadebfdr" does not appear to be a real word in the English language. It seems to be a random string of letters.'}